# Segmentação Final - Remoção de Marcas Circulares

Versão otimizada para remover marcas de medição circulares (pontos pretos do retículo).

## Algoritmo Melhorado:

### 1. Detecção de Círculos:
- **HoughCircles**: Detecta círculos perfeitos
- **Análise de circularidade**: Identifica formas aproximadamente circulares
- **Filtro de área**: Remove apenas objetos de tamanho específico (50-2000 pixels)

### 2. Características das Marcas:
- **Perfeitamente circulares**: Circularidade > 0.5
- **Completamente pretas**: Intensidade < 20
- **Tamanho médio**: 50-2000 pixels de área
- **Bordas definidas**: Contornos nítidos

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, IntSlider, Dropdown, Checkbox
from IPython.display import display

In [2]:
# Carregamento das imagens
data_dir = '../data'
supported_formats = ('.jpg', '.jpeg', '.png', '.tiff', '.tif')

image_files = []
if os.path.exists(data_dir):
    image_files = [f for f in os.listdir(data_dir) 
                   if f.lower().endswith(supported_formats)]

print(f"Encontradas {len(image_files)} imagens: {image_files}" if image_files else "Nenhuma imagem encontrada")

Encontradas 26 imagens: ['P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'PCB1.1_LK_Q5.jpg', 'P1A_T1_LD_Q5_100X.jpg', 'CNN_P1_L1_T1_M1_LG_Q8_150x.jpg', 'PCB1.1_LK_Q7.jpg', 'NIVEL 2 (BR1_1CAMP_LO_Q6_125X).jpg', 'PCB1.1_LM_.jpg', 'NIVEL 5 (P1D_T1_LI_Q11_63X).jpg', 'NIVEL 4 (P1D_T2_LJ_Q10_150X).jpg', 'P1A_T1_LC_Q6_100X.jpg', 'CNN_P1_L1_T1_M1_LH_Q3_63x.jpg', 'CNN_P1_L1_T1_M1_LI_Q2_50x.jpg', 'P1A_T1_LG_Q8_150X (1).jpg', 'PCB1.1.jpg', 'NIVEL 1 (BR3_1CAMP_LC_Q6_100X).jpg', 'NIVEL 3 (P1D_T3_LH_Q8_100X (2)).jpg', 'PCB1.1LB_Q9.jpg', 'CNN_P1_L1_T1_M1_LG_Q5_80x.jpg', 'PCB1.1_LK_Q10.jpg', 'CNN_P1_L1_T1_M1_LH_Q9_F1_150x.jpg', 'PCB1.1_LB_Q6.jpg', 'CNN_P1_L1_T1_M1_LC_Q9_100x.jpg', 'CNN_P1_L1_T1_M1_LG_Q11_100x.jpg', 'CNN_P1_L1_T1_M1_LI_Q6_125x.jpg', 'P1A_T1_LI_Q11_40X.jpg']


In [3]:
def remove_circular_marks(img_rgb, intensity_threshold=20, min_area=50, max_area=2000, min_circularity=0.5):
    """
    Remove marcas circulares pretas (pontos de medição/retículo)
    
    Parâmetros:
    - img_rgb: imagem RGB
    - intensity_threshold: limiar para detectar pixels pretos
    - min_area, max_area: faixa de área para marcas
    - min_circularity: circularidade mínima (0-1)
    
    Retorna:
    - cleaned_img: imagem com marcas removidas
    - marks_mask: máscara das marcas removidas
    """
    # Converte para escala de cinza
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    # Detecta pixels muito escuros
    black_mask = gray < intensity_threshold
    
    # Limpa a máscara
    kernel = np.ones((3,3), np.uint8)
    black_mask_clean = cv2.morphologyEx(black_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    
    # Encontra contornos
    contours, _ = cv2.findContours(black_mask_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Cria máscara para marcas circulares
    marks_mask = np.zeros(gray.shape, dtype=np.uint8)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        # Filtra por área (tamanho típico de marcas)
        if min_area < area < max_area:
            # Calcula circularidade
            perimeter = cv2.arcLength(contour, True)
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                
                # Se for circular o suficiente, marca para remoção
                if circularity > min_circularity:
                    cv2.fillPoly(marks_mask, [contour], 255)
    
    # Usa HoughCircles como método adicional
    circles = cv2.HoughCircles(255 - gray, cv2.HOUGH_GRADIENT, 1, 20,
                              param1=50, param2=15, minRadius=4, maxRadius=25)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            # Verifica se o círculo é realmente preto
            mask_circle = np.zeros(gray.shape, dtype=np.uint8)
            cv2.circle(mask_circle, (x, y), r, 255, -1)
            mean_intensity = cv2.mean(gray, mask_circle)[0]
            
            if mean_intensity < intensity_threshold + 10:  # Margem pequena
                cv2.circle(marks_mask, (x, y), r + 2, 255, -1)  # +2 para margem
    
    # Remove as marcas usando inpainting
    if np.any(marks_mask):
        cleaned_img = cv2.inpaint(img_rgb, marks_mask, 5, cv2.INPAINT_TELEA)
    else:
        cleaned_img = img_rgb.copy()
    
    return cleaned_img, marks_mask

In [4]:
def segment_microplastics_final(image_path, background_threshold=160, min_saturation=20, 
                              remove_marks=True, apply_morphology=True):
    """
    Segmentação final com remoção otimizada de marcas circulares
    """
    # Carrega imagem
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Remove marcas circulares se habilitado
    if remove_marks:
        processed_img, marks_mask = remove_circular_marks(img_rgb)
    else:
        processed_img = img_rgb.copy()
        marks_mask = np.zeros(img_rgb.shape[:2], dtype=np.uint8)
    
    # Segmentação HSV (mesmo algoritmo anterior)
    hsv = cv2.cvtColor(processed_img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    
    # Detecção de fundo
    r, g, b = cv2.split(processed_img)
    background_mask = ((r.astype(np.float32) + g.astype(np.float32) + b.astype(np.float32)) / 3) > background_threshold
    object_mask = ~background_mask
    
    # Detecção de cores
    blue_mask = object_mask & (h >= 100) & (h <= 130) & (s > min_saturation) & (v > 15)
    red_mask = object_mask & (((h >= 0) & (h <= 25)) | (h >= 170)) & (s > min_saturation) & (v > 20)
    green_mask = object_mask & (h >= 40) & (h <= 80) & (s > min_saturation) & (v > 20)
    yellow_mask = object_mask & (h >= 25) & (h <= 40) & (s > min_saturation) & (v > 30)
    dark_mask = object_mask & (v < 60) & (s < min_saturation)
    dark_mask = dark_mask & ~blue_mask & ~red_mask & ~green_mask & ~yellow_mask
    
    # Converte máscaras
    masks = {
        'azul': (blue_mask * 255).astype(np.uint8),
        'vermelho': (red_mask * 255).astype(np.uint8),
        'verde': (green_mask * 255).astype(np.uint8),
        'amarelo': (yellow_mask * 255).astype(np.uint8),
        'escuro': (dark_mask * 255).astype(np.uint8),
        'fundo': (background_mask * 255).astype(np.uint8),
        'marcas_removidas': marks_mask
    }
    
    # Limpeza morfológica
    if apply_morphology:
        kernel = np.ones((2,2), np.uint8)
        for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
            masks[color] = cv2.morphologyEx(masks[color], cv2.MORPH_OPEN, kernel)
            masks[color] = cv2.morphologyEx(masks[color], cv2.MORPH_CLOSE, kernel)
    
    # Segmentações
    segmentations = {}
    for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
        segmentations[color] = cv2.bitwise_and(processed_img, processed_img, mask=masks[color])
    
    # Contagem
    counts = {}
    min_area = 15
    for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
        contours, _ = cv2.findContours(masks[color], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        valid_contours = [c for c in contours if cv2.contourArea(c) > min_area]
        counts[color] = len(valid_contours)
    
    return img_rgb, processed_img, masks, segmentations, counts

In [5]:
def interactive_final_segmentation(image_name, background_threshold, min_saturation, 
                                 remove_marks, apply_morphology):
    """
    Interface final com remoção otimizada de marcas
    """
    if not image_files:
        print("Nenhuma imagem disponível")
        return
    
    image_path = os.path.join(data_dir, image_name)
    original, cleaned, masks, segmentations, counts = \
        segment_microplastics_final(image_path, background_threshold, min_saturation, 
                                  remove_marks, apply_morphology)
    
    # Visualização com 14 painéis
    plt.figure(figsize=(35, 10))
    
    # Linha superior
    plt.subplot(2, 7, 1)
    plt.imshow(original)
    plt.title('Original\n(Com Marcas)')
    plt.axis('off')
    
    plt.subplot(2, 7, 2)
    plt.imshow(cleaned)
    plt.title('Limpa\n(Marcas Removidas)' if remove_marks else 'Sem Limpeza')
    plt.axis('off')
    
    plt.subplot(2, 7, 3)
    plt.imshow(masks['marcas_removidas'], cmap='Reds')
    plt.title('Marcas Detectadas\n(Removidas)')
    plt.axis('off')
    
    # Máscaras de cores
    colors_info = [('azul', 'Blues', 'Azul'), ('vermelho', 'Reds', 'Vermelho'), 
                   ('verde', 'Greens', 'Verde'), ('amarelo', 'YlOrBr', 'Amarelo')]
    
    for i, (color, cmap, label) in enumerate(colors_info, 4):
        plt.subplot(2, 7, i)
        plt.imshow(masks[color], cmap=cmap)
        plt.title(f'{label}\n({counts[color]} objetos)')
        plt.axis('off')
    
    # Linha inferior - segmentações
    plt.subplot(2, 7, 8)
    plt.imshow(masks['escuro'], cmap='gray')
    plt.title(f'Escuro\n({counts["escuro"]} objetos)')
    plt.axis('off')
    
    seg_colors = ['azul', 'vermelho', 'verde', 'amarelo']
    seg_labels = ['Azuis', 'Vermelhos', 'Verdes', 'Amarelos']
    
    for i, (color, label) in enumerate(zip(seg_colors, seg_labels), 9):
        plt.subplot(2, 7, i)
        plt.imshow(segmentations[color])
        plt.title(f'Microplásticos\n{label}')
        plt.axis('off')
    
    # Gráfico final
    plt.subplot(2, 7, 13)
    bar_colors = ['blue', 'red', 'green', 'gold', 'black']
    bar_labels = ['Azul', 'Verm', 'Verde', 'Amar', 'Escuro']
    values = [counts[c] for c in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']]
    
    bars = plt.bar(bar_labels, values, color=bar_colors, alpha=0.7)
    plt.title('Contagem\nFinal')
    plt.ylabel('Objetos')
    plt.xticks(rotation=45, fontsize=8)
    
    for bar, value in zip(bars, values):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    str(value), ha='center', va='bottom', fontsize=8)
    
    # Comparação antes/depois
    plt.subplot(2, 7, 14)
    comparison = np.hstack([original[:, :original.shape[1]//2], 
                           cleaned[:, cleaned.shape[1]//2:]])
    plt.imshow(comparison)
    plt.title('Antes | Depois')
    plt.axis('off')
    
    total_objects = sum(values)
    marks_status = 'REMOVIDAS' if remove_marks else 'MANTIDAS'
    plt.suptitle(f'Segmentação Final - Marcas: {marks_status} | Total: {total_objects} microplásticos', 
                fontsize=16)
    
    plt.tight_layout()
    plt.show()

In [7]:
# Interface final otimizada
if image_files:
    interact(interactive_final_segmentation,
             image_name=Dropdown(options=image_files, description='Imagem:'),
             background_threshold=IntSlider(min=120, max=200, step=10, value=160, 
                                          description='Limiar Fundo:'),
             min_saturation=IntSlider(min=10, max=60, step=5, value=20, 
                                    description='Saturação Mín:'),
             remove_marks=Checkbox(value=True, 
                                 description='Remover Marcas Circulares'),
             apply_morphology=Checkbox(value=True, 
                                     description='Limpeza Morfológica'))
else:
    print("Adicione imagens ao diretório ../data/")

interactive(children=(Dropdown(description='Imagem:', options=('P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'P…